## Data Wrangling II:
### 1. Data Preparation
### 2. Encoding
### 3. Extra Reading

## 1. Data Preparation

### 1.1 Missing Data

In [ ]:
import pandas as pd
import numpy as np

For data with float64 dtype, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data.

In [ ]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

In [ ]:
float_data

The `isna` method gives us a Boolean Series with True where values are null:



In [ ]:
float_data.isna()

The built-in Python None value is also treated as NA:



In [ ]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

In [ ]:
string_data

In [ ]:
string_data.isna()

In [ ]:
float_data = pd.Series([1, 2, None], dtype='float64')

In [ ]:
float_data

In [ ]:
float_data.isna()

There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas `.isna` and Boolean indexing, `dropna` can be helpful. On a Series, it returns the Series with only the nonnull data and index values:

In [ ]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

In [ ]:
data

In [ ]:
data.dropna()

In [ ]:
data[data.notna()]

With DataFrame objects, there are different ways to remove missing data. You may want to drop rows or columns that are all `NA`, or only those rows or columns containing any `NA`s at all. `dropna` by default drops any row containing a missing value:

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

In [ ]:
data


In [ ]:
data.dropna()

Passing `how="all"` will drop only rows that are all `NA`:



In [ ]:
data.dropna(how="all")

In [ ]:
# Keep in mind that these functions return new objects by default and do not modify the contents 
# of the original object.
data

To drop columns in the same way, pass `axis="columns"`:

In [ ]:
data[4] = np.nan

In [ ]:
data

In [ ]:
data.dropna(axis="columns", how="all")

Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the thresh argument:

In [ ]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))


In [ ]:
df.iloc[:4, 1] = np.nan


In [ ]:
df.iloc[:2, 2] = np.nan


In [ ]:
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(thresh=2)

Calling fillna with a constant replaces missing values with a certain value:

In [ ]:
df.fillna(0)

The same interpolation methods available for reindexing can be used with fillna:



In [ ]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

In [ ]:
df.iloc[2:, 1] = np.nan

In [ ]:
df.iloc[4:, 2] = np.nan

In [ ]:
df

In [ ]:
df.fillna(method="ffill")

In [ ]:
df.fillna(method="ffill", limit=2)

With fillna you can do other things such as simple data imputation using the median or mean statistics:

In [ ]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

In [ ]:
data.fillna(data.mean())

### 1.2 Data Transformation

#### Removing Duplicates

In [ ]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                         "k2": [1, 1, 2, 3, 3, 4, 4]})

In [ ]:
data

In [ ]:
data.duplicated()

`drop_duplicates` returns a DataFrame with rows where the duplicated array is `False` filtered out:



In [ ]:
data.drop_duplicates()

Both methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates based only on the "k1" column:

In [ ]:
data["v1"] = range(7)

In [ ]:
data

In [ ]:
data.drop_duplicates(subset=["k1"])

`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep="last"` will return the last one:

#### Transforming data with functions

Consider the following data collected about various kinds of meat:

In [ ]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [ ]:
data

Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [ ]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

The `map` method on a Series accepts a function or dictionary-like object containing a mapping to do the transformation of values:

In [ ]:
data["animal"] = data["food"].map(meat_to_animal)

In [ ]:
data

We could also have passed a function that does all the work:

In [ ]:
def get_animal(x):
    return meat_to_animal[x]

In [ ]:
data["food"].map(get_animal)

#### Replacing Values

Filling in missing data with the `fillna` method is a special case of more general value replacement. `map` can be used to modify a subset of values in an object, but replace provides a simpler and more flexible way to do so. Let’s consider this Series:

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [ ]:
data

The `-999` values might be an indicator of values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series:



In [ ]:
data.replace(-999, np.nan)

If you want to replace multiple values at once, you instead pass a list and then the substitute value:


In [ ]:
data.replace([-999, -1000], np.nan)

To use a different replacement for each value, pass a list of substitutes:



In [ ]:
data.replace([-999, -1000], [np.nan, 0])

The argument passed can also be a dictionary:



In [ ]:
data.replace({-999: np.nan, -1000: 0})

## 2. Encoding

### 2.1 Categorical Encoding

The first case of categories is about data discretization. Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use `pandas.cut`:

In [ ]:
bins = [18, 25, 35, 60, 100]

In [ ]:
age_categories = pd.cut(ages, bins)

In [ ]:
age_categories

The object pandas returns is a special Categorical object. The output you see describes the bins computed by `pandas.cut`. Each bin is identified by a special (unique to pandas) interval value type containing the lower and upper limit of each bin:

In [ ]:
age_categories.codes

In [ ]:
age_categories.categories

In [ ]:
age_categories.categories[0]

In [ ]:
pd.value_counts(age_categories)

`pd.value_counts(categories)` are the bin counts for the result of `pandas.cut`.

### 2.2 One-Hot Encoding

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a dummy or indicator matrix, also known as one-hot encoding. 
If a column in a DataFrame has `k` distinct values, you would derive a matrix or DataFrame with `k` columns containing all 1s and 0s. pandas has a `pandas.get_dummies` function for doing this.

In [ ]:
# For example de DF:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                       "data1": range(6)})

In [ ]:
df

In [ ]:
pd.get_dummies(df["key"])

If a row in a DataFrame belongs to multiple categories, we have to use a different approach to create the dummy variables. For example, in the MovieLens 1M dataset:

In [ ]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("movies.dat", sep="::",
                           header=None, names=mnames, engine="python")

In [ ]:
movies[:10]

pandas has a special Series method `str.get_dummies` that handles multiple group membership encoded as a delimited string, in this case, `|` for the genres:

In [ ]:
dummies = movies["genres"].str.get_dummies("|")

In [ ]:
dummies.iloc[:10, :6]

### 2.3 Text representation

Suppose that you have a dataset with sentences like comments in a social media. A way to represent text as numeric data is using one-hot encoding using the words as categories. However, there are another approaches that could yield to better performance when training a Machine Learning model.

#### Bag Of Words
The first alternative approach to one-hot-encoding is the Bag of Words (BoW) model. Suppose you have the next toy dataset with 4 sentences:

`doc = ['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']`

If we map the words with IDs as: dog = 1, bites = 2, man = 3, meat = 4 , food = 5, eats = 6
Then the one-hot encoding scheme for the first sentence would be

`[[1 0 0 0 0 0], [0 1 0 0 0 0], [0 0 1 0 0 0]]`

The BoW model counts the frequencies of words in a sentence assigning the total of counts to the IDs. Thus, in the BoW model, the first sentence representantion can be stated as:

`[1 1 1 0 0 0]`.

This is because the first three words in the vocabulary appeared exactly once, and the last three did not appear at all.

#### scikit-learn API
In this and the next section we are going to use the sklearn package for the transformation of data, namely, text representation and scalings.

In [ ]:
doc = ['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [ ]:
#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(doc)

In [ ]:
#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

In [ ]:
#See the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'man bites dog: ",bow_rep[1].toarray())

In [ ]:
#Get the representation using this vocabulary, for a new text
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':",
temp.toarray())

Notice that 'dog' is the only word considered when counting because of its presence in the original voabulary.

Sometimes, we don’t care about the frequency of occurrence of words in text and we only want to represent whether a word exists in the text or not. In this case, use the `binary` argument set to `True`

In [ ]:
count_vect = CountVectorizer(binary=True)
bow_rep_bin = count_vect.fit_transform(doc)
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())

#### TF-IDF

`TF-IDF`, or term frequency–inverse document frequency, quantify the importance of a given word relative to other words in the document and in the corpus. 

`TF` (term frequency) measures how often a term or word occurs in a given document. Since different documents in the corpus may be of different lengths, a term may occur more often in a longer document as compared to a shorter document. To normalize these counts, we divide the number of occurrences by the length of the document. `IDF` (inverse document frequency) measures the importance of the term across a corpus. This solves the problem of common stop words like, is, are, am, etc. Now let's see a TF-IDF implementation:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(doc)
print(tfidf.idf_) #IDF for all words in the vocabulary
print(tfidf.get_feature_names()) #All words in the vocabulary.

In [ ]:
temp = tfidf.transform(["dog and man are friends"])
print("Tfidf representation for 'dog and man are friends':\n", temp.toarray())

### 2.4 Feature Scaling

Let's assume that we have two features where one feature is measured on a scale from 1 to 10 and the second feature is measured on a scale from 1 to 100,000, respectively. Some ML algorithms would fail to converge or would too long to converge. Now, there are two common approaches to bringing different features onto the same
scale: normalization and standardization. Normalization refers to the rescaling of the features to a range of `[0, 1]`, which is a special case of min-max scaling.

Using standardization, we center the feature columns at mean 0 with standard deviation 1 so that the feature  columns have the same parameters as a standard normal distribution (zero mean and unit variance).

We illustrate whis with the popular iris dataset:

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
data = load_iris()

In [ ]:
X = data['data']

Let's use the sklearn MinMaxScaler for normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mms = MinMaxScaler()

In [ ]:
X_norm = mms.fit_transform(X)

In [ ]:
X[:10]

In [ ]:
X_norm[:10]

Similar to the MinMaxScaler, let's use the sklearn StandardScaler for standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
stdsc = StandardScaler()

In [ ]:
X_std = stdsc.fit_transform(X)

In [ ]:
X_std[:10]

## 3. Extra Reading

### 3.1 sklearn tranformation pipelines
#### Suggested Reading: https://scikit-learn.org/stable/modules/compose.html#pipeline

### 3.2 Feature engineering
#### Suggested Reading: https://www.analyticsvidhya.com/blog/2021/03/step-by-step-process-of-feature-engineering-for-machine-learning-algorithms-in-data-science/